In [ ]:
import zipfile2
from zipfile2 import ZipFile
import os
import string
import json
from IPython.display import Audio

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import librosa

import sklearn

import tensorflow as tf
import keras


In [ ]:
# extracting the files
# base_path = '../dataset/NOIZEUS_dataset/'
# for file in os.listdir(base_path):
#     if file.endswith('.zip'):
#         path = f'{base_path}/{file}'
#         filename = file.split('.')[0]
#         if not os.path.exists(f'{base_path}/{filename}'):
#             os.makedirs(f'{base_path}/{filename}')
#         with ZipFile(path , 'r') as zip:
#             zip.extractall(path = f'{base_path}/{filename}')
#         if os.path.exists(path):
#             os.remove(path)

In [ ]:
# preparing the aurora-4 dataset
base_path = '../dataset/NOIZEUS_dataset/'
audio_file_paths = []
db_array = ['0dB' , '5dB' , '10dB' , '15dB']
place_array = ['airport' , 'babble' , 'car' , 'exhibition' , 'restaurant' , 'station' , 'street' , 'train']
for place in place_array:
    for dB in db_array:
        path = f'{base_path}/{place}_{dB}/{dB}'
        audio_file_paths.append(path)

In [ ]:
noizeus_dataset = pd.DataFrame({'path' : audio_file_paths})
noizeus_dataset.to_csv('datasets/noizeus_dataset_audio_file_paths.csv')

In [10]:
# making a dataset of spectrograms of files
for folder_path in noizeus_dataset['path']:
    for file_path in os.listdir(folder_path):
        file_path = f'{folder_path}/{file_path}'
        y, sr = librosa.load(file_path,sr=22050)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=60)
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mfccs, sr=sr, x_axis='time')
        plt.colorbar()
        plt.title("MFCC")
        plt.show()
        plt.close()

In [45]:
filenames = []
mel_spectograms = []
try:
    for folder_path in noizeus_dataset['path']:
        for filename in os.listdir(folder_path):
            file_path = f'{folder_path}/{filename}'
            filenames.append(filename)
            y, sr = librosa.load(file_path,sr=22050)
            mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
            mel_spectograms.append(mel_spec)
            # Convert to decibels for better visualization
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

            # Plot the Mel Spectrogram
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(mel_spec_db, sr=sr, hop_length=512, x_axis='time', y_axis='mel', cmap='viridis')
            plt.colorbar(format='%+2.0f dB')
            plt.title('Mel Spectrogram')
            plt.tight_layout()
            filename = filename.split('.')[0]
            plt.savefig(f'datasets/mel_spectogram_noizeus_dataset/{filename}.png' , bbox_inches = 'tight')
            # plt.show()
            plt.close()
except Exception as e:
    print(e)

[WinError 3] The system cannot find the path specified: '../dataset/NOIZEUS_dataset//train_15dB/15dB'


In [46]:
noisy_audiofiles_spectogram_dataset = pd.DataFrame({'noisy_file' : filenames , 'mel_spectorgrams' : mel_spectograms})
noisy_audiofiles_spectogram_dataset.to_csv('datasets/noisy_audiofiles_melspectograms_noizeus.csv',index=False)

In [44]:
filenames = []
mel_spectograms = []
clean_audio_path = '../dataset/NOIZEUS_dataset/clean/clean'
for file in os.listdir(clean_audio_path):
    filenames.append(file)
    path = f'{clean_audio_path}/{file}'
    y , sr = librosa.load(path , sr=22050)
    mel_spec = librosa.feature.melspectrogram(y = y , n_fft=2048 , hop_length = 512 , n_mels = 128)
    mel_spectograms.append(mel_spec)

    # plotting spectrum
    mel_spec_db = librosa.power_to_db(mel_spec , ref=np.max)
    plt.figure(figsize=(10,4))
    librosa.display.specshow(mel_spec_db , sr = 22050, hop_length=512,n_fft=2048,x_axis='time' , y_axis='mel')
    plt.colorbar(format = '%+2.0f dB')
    plt.title('Mel Spectogram')
    plt.tight_layout()
    filename = file.split('.')[0]
    plt.savefig(f'datasets/mel_spectogram_noizeus_dataset/{filename}.png' , bbox_inches = 'tight')
    plt.close()

In [47]:
clean_audiofiles_spectogram_dataset = pd.DataFrame({'noisy_file' : filenames , 'mel_spectorgrams' : mel_spectograms})
clean_audiofiles_spectogram_dataset.to_csv('datasets/clean_audiofiles_melspectograms_noizeus.csv',index=False)

In [ ]:
# defining a neural network
encoder = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(100 , activation = 'relu'),
        tf.keras.layers.Dense(30 , activation = 'relu')
    ]
)
